In [2]:
import sys
sys.path.insert(1, '../../Src')
from utils.preprocessing import *
import training

import numpy as np
import pandas as pd
import os

import glob
from math import ceil
from sklearn.metrics import mean_squared_error


warnings.filterwarnings('ignore')

PATH_DATASET = '../../dataset/'
PATH_RESULTS = '../../results/Demanda/'


FILL = False
TARGET = 'Demanda'


N_FOLDS = 3
N_FEATURE_IMPORTANCE = 15

plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
SEED = 42
seed_everything(SEED)
training.PATH_RESULTS = PATH_RESULTS
training.submission_analysis.TARGET  = TARGET

%load_ext autoreload
%autoreload 2

In [3]:
submission_top_reference = 'Submission_73.csv'
print('reading .. ',os.path.join('../../results/',submission_top_reference))
result = pd.read_csv(os.path.join('../../results/',submission_top_reference))

reading ..  ../../results/Submission_73.csv


In [4]:
result.head()

,ID,Demanda
0,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0
1,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0
2,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0
3,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0
4,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0


In [6]:
descriptive_columns = ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']
reverse_mapping_file = '../../utils/reverse_dict_mapping_list.txt'

result[['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']] = result['ID'].str.split('|',expand=True)

with open(reverse_mapping_file, 'rb') as f:
    reverse_mapping = pickle.load( f)
    
descriptive_columns = ['Z_MARCA', 'Z_GAMA', 'Z_MODELO',
                       'Z_DEPARTAMENTO', 'Z_PUNTO_VENTA']
i=0
for column in descriptive_columns:
    if column in ['Z_MODELO','Z_PUNTO_VENTA','Z_GAMA','Z_WEEK']:
        inv_reverse_mapping = {v: k for k, v in reverse_mapping[i].items()}

        result[column] = result[column].map(inv_reverse_mapping)
    i+=1
    
result = result.rename(columns = {TARGET: TARGET+'_real'})

In [7]:
result.head()

,ID,Demanda_real,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK
0,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_51
1,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_52
2,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_53
3,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_54
4,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_55


In [8]:
result['Demanda_pred'] = 0

WEEKS_LIST = {}

TOTAL_FILES = 0
for week in range(1,3):
    for idx,model_type in enumerate(['xgboost']):
        for model in ['c_model_v3','c_model_v2']:
            path_folder = os.path.join(PATH_RESULTS,model_type,model,'shift_'+str(week))
            list_filename_csv = glob.glob(os.path.join(path_folder,"*_submission.csv"))
            if len(list_filename_csv)>0:
                score = float(list_filename_csv[0].split('rmse_cv_test_')[-1].split('_')[0])
                
                if score>(2.15+week*50):#*0.15):
                    break
                print('reading ..',list_filename_csv[0])
                TOTAL_FILES+=1
                filename_csv = list_filename_csv[0]
                pred = pd.read_csv(filename_csv).rename(columns={"Demanda":"Demanda_pred"}).reset_index(drop=True)
                SEMANA = 'SEMANA_'+str(50+week)
                
                if SEMANA in WEEKS_LIST.keys():
                    WEEKS_LIST[SEMANA]+=1.0
                else:
                    WEEKS_LIST[SEMANA]=1.0
                
                sub_group = result[["Z_MODELO","Z_PUNTO_VENTA","Z_GAMA","Z_WEEK"]][result['Z_WEEK']==SEMANA].merge(
                    pred[["Z_MODELO","Z_PUNTO_VENTA","Z_GAMA","Z_WEEK","Demanda_pred"]],how='left').reset_index(drop=True)
                
                result['Demanda_pred'][result['Z_WEEK']==SEMANA] += sub_group['Demanda_pred'].values
print('TOTAL_FILES = ',TOTAL_FILES)

reading .. ../../results/Demanda/xgboost/c_model_v3/shift_1/rmse_cv_train_2.6648_rmse_cv_test_2.2759_submission.csv
reading .. ../../results/Demanda/xgboost/c_model_v2/shift_1/rmse_cv_train_2.5863_rmse_cv_test_2.161_submission.csv
reading .. ../../results/Demanda/xgboost/c_model_v3/shift_2/rmse_cv_train_3.0301_rmse_cv_test_2.6724_submission.csv
reading .. ../../results/Demanda/xgboost/c_model_v2/shift_2/rmse_cv_train_2.9967_rmse_cv_test_2.5806_submission.csv
TOTAL_FILES =  4


In [9]:
for SEMANA in WEEKS_LIST:
    result['Demanda_pred'][result['Z_WEEK']==SEMANA] /= WEEKS_LIST[SEMANA]

In [10]:
SEMANA

'SEMANA_52'

In [11]:
result.head()

,ID,Demanda_real,Z_MODELO,Z_PUNTO_VENTA,Z_GAMA,Z_WEEK,Demanda_pred
0,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_51,0.010752
1,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_52,0.015741
2,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_53,0.000000
3,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_54,0.000000
4,009e0874cc07b2180b2b2b1039321041a1f42e66693c49...,0.0,MOD_170,PVENT_259,GAM_2,SEMANA_55,0.000000


In [12]:
result['Demanda_pred'][~result['Z_WEEK'].isin(WEEKS_LIST.keys())]  = result['Demanda_real'][~result['Z_WEEK'].isin(WEEKS_LIST.keys())] 

In [13]:
result['Demanda_pred'][result['Z_WEEK']=='SEMANA_52']  = 0.2*result['Demanda_pred'][result['Z_WEEK']=='SEMANA_52'] + 0.8*result['Demanda_real'][result['Z_WEEK']=='SEMANA_52']

In [17]:
rmse_target = mean_squared_error(result[TARGET+'_pred'],result[TARGET+'_real'], squared=False)

submissions_path = os.path.join(PATH_RESULTS,'submissions','submission_weeks2_repz12_0.2_pred_0.8_real_rmse_kaggle_'+str(np.round(rmse_target,3))+'.csv')
print('saving submission ..',submissions_path)

print('Final score referencial mean_squared_error')
print('RMSE Score referencial TARGET:',rmse_target)

if True:
    print('saving ..')
    result[['ID','Demanda_pred']].rename(columns={"Demanda_pred":"Demanda"}).to_csv(submissions_path,index=None)
    print('saved ! ')

saving submission .. ../../results/Demanda/submissions/submission_weeks2_repz12_0.2_pred_0.8_real_rmse_kaggle_0.412.csv
Final score referencial mean_squared_error
RMSE Score referencial TARGET: 0.411640722476265
saving ..
saved ! 


In [18]:
# 0.411640722476265